<a href="https://colab.research.google.com/github/TnTerry/StockDirection/blob/main/Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数据创建


In [ ]:
!pip install tushare

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

import tushare as ts
ts.set_token('217269f8e0e67c7db7d4da55a552dd2de030df524cd4639ed53a224d')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 合并新闻数据

只需要首次运行，后续可以直接跳过

分别将东方证券（eastmoney）和新浪网（sina）的新闻数据批量导入，并合并成为dataframe格式

In [ ]:
news_file_east_money = glob(r"*_news_data_eastmoney.csv")
news_file_sina = glob(r"*_news_data_sina.csv")

In [ ]:
file_name = news_file_east_money[0]
east_money_df = pd.read_csv(file_name)

for file_name in tqdm(news_file_east_money[1:]):
  df_i = pd.read_csv(file_name)
  east_money_df = pd.concat([east_money_df, df_i])

east_money_df.reset_index(inplace = True)
east_money_df.dropna(inplace = True)

100%|██████████| 4/4 [00:06<00:00,  1.60s/it]


In [ ]:
file_name = news_file_sina[0]
sina_df = pd.read_csv(file_name)

for file_name in tqdm(news_file_sina[1:]):
  df_i = pd.read_csv(file_name)
  sina_df = pd.concat([sina_df, df_i])

sina_df.reset_index(inplace = True)
sina_df = sina_df.loc[:,['datetime','content']]
sina_df.head()

100%|██████████| 4/4 [00:08<00:00,  2.03s/it]


,datetime,content
0,2018-10-08 23:59:42,恐慌指数VIX报18.38，触及6月28日以来日内最高点。
1,2018-10-08 23:49:13,道指跌超200点。
2,2018-10-08 23:44:58,道指跌150点。
3,2018-10-08 23:40:13,富时泛欧绩优300指数初步收跌1.13%。
4,2018-10-08 23:40:13,富时泛欧绩优300指数初步收跌1.13%。


In [ ]:
east_money_df = east_money_df.loc[:,['datetime','title']]
east_money_df.columns = ['datetime','content']

In [ ]:
sina_df.dropna(inplace=True)

In [ ]:
east_money_df.shape, sina_df.shape

((556808, 2), (1021875, 2))

In [ ]:
total_news_df = pd.concat([east_money_df, sina_df]).reset_index(drop=True)
total_news_df.head()

,datetime,content
0,2018-10-25 23:59:17,品钛美股上市首日开盘价14.35美元 较发行价涨21%
1,2018-10-25 23:45:08,国内商品期货夜盘涨跌互现 焦煤涨近2%
2,2018-10-25 23:35:07,在岸人民币兑美元(CNY)收报6.9499元
3,2018-10-25 23:30:14,王岐山访问以色列
4,2018-10-25 23:19:17,农业农村部公布第一批农药登记试验单位认定名单


## 新闻数据清洗

提取新闻中股票对应的名称和代码，未提取出来的赋值为UNK。

只需要首次运行，后续可以直接跳过

输出文件路径：/content/drive/MyDrive/Thesis/Data/east_money_df_code.hdf, key = news_data

In [ ]:
pro = ts.pro_api()

In [ ]:
pro = ts.pro_api()
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date', market = "主板")

In [ ]:
(data['ts_code'].apply(lambda x: x[-2:])).unique()

array(['SZ', 'SH'], dtype=object)

In [ ]:
import re

# 使用apply方法和re.sub函数剔除英文字母和数字
pattern = re.compile(r'[^\u4e00-\u9fa5]')
data['name'] = data['name'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
#
company_names = data['name'].to_list()

# 将公司名字列表转换为正则表达式
regex = '(' + '|'.join(company_names) + ')'

# 使用str.contains()函数匹配新闻标题中是否包含上市公司名字
mask = total_news_df['content'].str.contains(regex)

# 使用where()函数保留包含上市公司名字的行，并设置其他行的值为NaN
df_filtered = total_news_df.where(mask).dropna()
# df_filtered = east_money_df.where(mask).dropna()

# 使用split()函数提取上市公司名字，并将其作为一列添加到df_filtered中
df_filtered['company_name'] = df_filtered['content'].str.extract('(' + regex + ')', expand=False)[0]

In [ ]:
df_filtered = df_filtered.reset_index(drop=True)
if "index" in df_filtered.columns:
  df_filtered.drop("index",axis = 1, inplace = True)

In [ ]:
df_filtered.shape

(109601, 3)

In [ ]:
remove_name_lst = ['科技','未来','交投','洲际', '新动力', '曙光', '农产品']
df_filtered = df_filtered[~df_filtered['company_name'].isin(remove_name_lst)].reset_index(drop=True)
df_filtered.shape

(106090, 3)

In [ ]:
df_filtered.to_hdf("news_df.hdf", key = "news_data")

In [ ]:
seq_len = 10
return_interval = 1
freq = "D"

key = f"seq_len_{seq_len}_return_interval_{return_interval}_freq_{freq}"
tmp_df = pd.read_hdf("stock_news_sampled_df.hdf", key = key)

In [ ]:
int(0.8*len(tmp_df))

24356

In [ ]:
tmp_df.iloc[int(0.8*len(tmp_df)),:]

datetime                                               20220520
title                                       ST花王：法院裁定受理控股股东重整申请
code                                                  603007.SH
name                                                         花王
stock_data    {'open': [3.48, 3.3, 3.34, 3.3, 3.36, 3.49, 3....
Name: 24356, dtype: object

In [ ]:
tmp_df

,datetime,title,code,name,stock_data
0,20181025,科大讯飞：翻译机上线5个月销量超20万台,002230.SZ,科大讯飞,"{'open': [22.5, 21.67, 22.94, 23.58, 21.8, 20...."
1,20181025,意大利副总理迪·迈耶：相信意大利/德国国债利差将在未来几周收窄,600532.SH,未来,"{'open': [4.59, 4.39, 4.55, 4.42, 4.43, 4.15, ..."
2,20181025,光大证券：片仔癀产品未来提价空间大,601788.SH,光大证券,"{'open': [9.82, 9.1, 9.15, 9.78, 8.68, 7.98, 8..."
3,20181025,珠海中富股份拍卖尚无成交进展,000659.SZ,珠海中富,"{'open': [3.11, 3.02, 3.16, 3.2, 3.0, 2.9, 2.8..."
4,20181025,离开任志强的华远地产要凉凉？标普将评级修订为负面,600743.SH,华远地产,"{'open': [2.13, 2.05, 2.04, 2.08, 2.0, 1.94, 2..."
...,...,...,...,...,...
30440,20221230,游戏股开盘走强 惠程科技直线拉升涨停,002168.SZ,惠程科技,"{'open': [3.88, 3.71, 3.78, 3.8, 3.86, 3.88, 3..."
30441,20221230,金杯汽车：金杯车辆被裁定破产重整,600609.SH,金杯汽车,"{'open': [4.18, 4.13, 4.15, 4.22, 4.23, 4.16, ..."
30442,20221231,Nreal Air第10万台量产下线 领益智造承接其整机组装等业务,002600.SZ,领益智造,"{'open': [4.56, 4.55, 4.52, 4.57, 4.56, 4.52, ..."
30443,20221231,2022年元宇宙概念股年终汇总：福石控股领涨60.37% 歌尔股份领跌68.72%,300071.SZ,福石控股,"{'open': [4.4, 3.69, 3.75, 3.8, 3.85, 3.91, 3...."


## 股票数据

只需要首次运行即可，后续可以直接跳过

输出文件路径：/content/drive/MyDrive/Thesis/Data/stock_data.hdf

不同参数的数据根据不同的key进行导出：seq_len{seq_len}\_return\_interval_{return_interval}\_freq_{freq}

In [ ]:
news_df = pd.read_hdf("/content/drive/MyDrive/Thesis/Data/news_df.hdf", key = "news_data")
news_df.head()

,datetime,content,company_name
0,2018-10-25 21:55:02,龙虎榜解读：机构净买这3股 一游资4800万买南京证券,南京证券
1,2018-10-25 21:22:13,伊力特前三季度净利增逾一成 养老保险基金现身股东榜,伊力特
2,2018-10-25 21:00:28,今日北向资金流出近34亿元 贵州茅台净流出超9亿元,贵州茅台
3,2018-10-25 20:54:07,南京证券、西南证券等券商股连日大涨 为各路游资炒作,南京证券
4,2018-10-25 20:24:29,金科股份：融创中国合计持股超过实控人黄红云及其一致行动人持有的股份数量,金科股份


In [ ]:
news_df.shape

(280627, 3)

超过15:00的即被归为第二天的新闻

In [ ]:
import datetime
news_df['Date'] = news_df['datetime'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
news_df['Date'] = news_df['Date'].apply(lambda x: x+datetime.timedelta(days = 1) if x.time()>datetime.time(15, 0, 0)\
    else x)
news_df['Date'] = news_df['Date'].apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
news_df['Date'] = news_df['Date'].apply(lambda x: x.split(" ")[0]) # 去除datetime中的时间部分
news_df['Date'] = news_df['Date'].str.replace("-","")
news_df.head()

,datetime,content,company_name,Date
0,2018-10-25 21:55:02,龙虎榜解读：机构净买这3股 一游资4800万买南京证券,南京证券,20181026
1,2018-10-25 21:22:13,伊力特前三季度净利增逾一成 养老保险基金现身股东榜,伊力特,20181026
2,2018-10-25 21:00:28,今日北向资金流出近34亿元 贵州茅台净流出超9亿元,贵州茅台,20181026
3,2018-10-25 20:54:07,南京证券、西南证券等券商股连日大涨 为各路游资炒作,南京证券,20181026
4,2018-10-25 20:24:29,金科股份：融创中国合计持股超过实控人黄红云及其一致行动人持有的股份数量,金科股份,20181026


In [ ]:
pro = ts.pro_api()
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name', market = "主板")

import re

# 使用apply方法和re.sub函数剔除英文字母和数字
pattern = re.compile(r'[^\u4e00-\u9fa5]')
data['name'] = data['name'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
data.columns = ['ts_code', 'symbol', 'company_name']

In [ ]:
news_df = pd.merge(news_df, data, on = 'company_name')

In [ ]:
news_df.head()

,datetime,content,company_name,Date,ts_code,symbol
0,2018-10-25 21:55:02,龙虎榜解读：机构净买这3股 一游资4800万买南京证券,南京证券,20181026,601990.SH,601990
1,2018-10-25 20:54:07,南京证券、西南证券等券商股连日大涨 为各路游资炒作,南京证券,20181026,601990.SH,601990
2,2019-01-06 21:27:25,南京证券：A股将迎来较好战略布局期,南京证券,20190107,601990.SH,601990
3,2019-06-13 09:26:37,南京证券今日巨量解禁 开盘大跌9%,南京证券,20190613,601990.SH,601990
4,2019-06-17 07:49:58,南京证券上市一年：经纪业务靠天吃饭 投行业务待提高,南京证券,20190617,601990.SH,601990


In [ ]:
del data

使用tushare接口提取股票在对应及新闻发生后的日线走势数据。

<!-- lookback为回溯期，即使用lookback天的数据进行预测 -->

seq_len：数据的长度，即使用seq_len天的数据进行预测

return_interval：收益率的计算区间长度

freq：数据频率

In [ ]:
pro = ts.pro_api()
TradeDay_df = pro.trade_cal(exchange='SSE', start_date='20180101', end_date='20230401')
TradeDay_df.head()

,exchange,cal_date,is_open,pretrade_date
0,SSE,20230401,0,20230331
1,SSE,20230331,1,20230330
2,SSE,20230330,1,20230329
3,SSE,20230329,1,20230328
4,SSE,20230328,1,20230327


In [ ]:
ttl_tradeday_lst = TradeDay_df.loc[TradeDay_df['is_open']==1,'cal_date'].to_list()

In [ ]:
import time

seq_len = 10
return_interval = 1
freq = "D"
ma_lst = [5, 10, 22]

def get_stock_data_factor(# date, stock_code,
              item,
              freq=freq, seq_len = seq_len, ma_lst = ma_lst,
              return_interval = return_interval):
  '''
  带有基础因子的数据
  date：新闻发布的日期
  freq：数据频率
  '''
  # date = item[1]
  # stock_code = item[3]
  idx, row = item

  date = row['Date']
  stock_code = row['ts_code']
  news_date = date

  # 找到日期前的第一个交易日
  while not TradeDay_df.loc[TradeDay_df['cal_date']==date,"is_open"].values[0]:
    index = TradeDay_df.loc[TradeDay_df['cal_date']==date,"is_open"].index+1
    date = TradeDay_df.loc[index,"cal_date"].values[0]

  # flag = ttl_tradeday_lst.index(date)
  # tradeday_lst = ttl_tradeday_lst[flag:(flag+seq_len+return_interval+1)]
  # ma_date_lst = ttl_tradeday_lst[flag:(flag+seq_len+return_interval+max(ma_lst)+1)]

  # 确定交易日范围
  tradeday_lst = [date]
  tradeday_filter = (TradeDay_df['pretrade_date']==date) & (TradeDay_df['is_open']==1)
  tradeday_lst.append(TradeDay_df['cal_date'][tradeday_filter].values[0]) # 新闻发生后的一个交易日作为计算return的节点

  while len(tradeday_lst)<(seq_len+return_interval):
    predate = TradeDay_df.loc[TradeDay_df['cal_date']==date,"pretrade_date"].values[0] # 找到date的前一个交易日
    tradeday_lst.insert(0,predate)
    date = predate

  # 设置ma_date_lst以计算均线
  ma_date_lst = tradeday_lst.copy()
  while len(ma_date_lst)<(seq_len+return_interval+max(ma_lst)):
    predate = TradeDay_df.loc[TradeDay_df['cal_date']==date,"pretrade_date"].values[0] # 找到date的前一个交易日
    ma_date_lst.insert(0,predate)
    date = predate
  start_date, end_date = ma_date_lst[0], ma_date_lst[-1]

  # 调用API获取数据
  time.sleep(0.13)
  stock_df = ts.pro_bar(ts_code=stock_code, start_date=start_date,
              end_date=end_date, factors=['tor', 'vr'], ma = ma_lst)
  # stock_df = stock_df[stock_df['trade_date'].isin(tradeday_lst)].reset_index(drop=True)

  # if idx%100==0:
  #   print("-"*20+f"{100*idx/len(news_df):.3f}% completed"+"-"*20)

  # 将所需要的序列数据提取出来
  # pct_chg作为Label只需要最后一个值；其余作为序列数据截断至倒数第二个
  try:
    res_dict = stock_df.to_dict(orient="list")
    return res_dict
  except:
    print(f"Error occured on on {news_date} for {stock_code}")
    return None


In [ ]:
from tqdm.notebook import tqdm
from multiprocessing import Pool
from functools import partial


with Pool(8) as pool:
  result = list(tqdm(pool.imap(get_stock_data_factor, news_df.iterrows()),total=len(news_df)))

# for pick_date in tqdm(news_df['Date'].unique()):
#   tmp_df = news_df[news_df['Date']==pick_date].reset_index()
#   with Pool(8) as pool:
#     result = pool.map(get_stock_data_factor, tmp_df.iterrows())
#   ttl_result.extend(result)

  0%|          | 0/280166 [00:00<?, ?it/s]

In [ ]:
len(result)

280166

In [ ]:
news_df['stock_data'] = pd.Series(result)
filename = "/content/drive/MyDrive/Thesis/Data/stock_news_df.hdf"
key = f"total_data"
news_df.to_hdf(filename, key = key)

<ipython-input-115-174455f9767e>:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['datetime', 'content', 'company_name', 'Date', 'ts_code', 'symbol',
       'stock_data'],
      dtype='object')]

  news_df.to_hdf(filename, key = key)
